In [1]:
import numpy as np
import pandas as pd
import time
import tkinter as tk
from openpyxl import load_workbook
import undetected_chromedriver as uc
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from tabulate import tabulate
from webdriver_manager.chrome import ChromeDriverManager
servico = Service(ChromeDriverManager().install())
import funcoes as fc

In [ ]:
#INICIA O NAVEGADOR
navegador = uc.Chrome(service=servico)
navegador.maximize_window()
navegador.get('https://sei.anatel.gov.br/')


In [ ]:
#FUNCAO QUE RECEBE USUARIO E SENHA
def preencher_campos():
    user = user_var.get()
    senha = senha_var.get()

    navegador.find_element(By.XPATH,'//*[@id="txtUsuario"]').send_keys(f"{user}")
    time.sleep(1)
    navegador.find_element(By.XPATH,'//*[@id="pwdSenha"]').send_keys(f"{senha}")
    root.destroy()

#INICIA JANELA
root = tk.Tk()
root.title("Login de usuário")
root.attributes('-topmost', True)
largura_tela = root.winfo_screenwidth()
root.geometry(f'+{largura_tela}+0')
root.state('zoomed')

#RECEBE USUARIO E SENHA
user_var = tk.StringVar()
senha_var = tk.StringVar()

frame = tk.Frame(root)
frame.place(relx=0.5, rely=0.5, anchor='center')

# CRIA LABEL CENTRALIZADO
tk.Label(frame, text="LOGIN NO SEI ANATEL", font=('Arial',30)).grid(row=0, columnspan=2)
tk.Label(frame, text="").grid(row=1, column=0)
tk.Label(frame, text="").grid(row=2, column=0)

#CRIA INPUT USUARIO
tk.Label(frame, text="Usuário:", font=('Arial',15)).grid(row=3, column=0, sticky='e')
tk.Entry(frame, textvariable=user_var,width=30).grid(row=3, column=1, sticky='w')

#CRIA INPUT SENHA CENSURADA
tk.Label(frame, text="Senha:", font=('Arial',15)).grid(row=4, column=0, sticky='e')
tk.Entry(frame, textvariable=senha_var, show="*",width=30).grid(row=4, column=1, sticky='w')

#BOTAO QUE ENVIA USUARIO E SENHA
tk.Label(frame, text="").grid(row=5, column=0)
tk.Button(frame, text="Entrar", font=('Arial',10), command=preencher_campos, width=20).grid(row=6, columnspan=2)

#INFORMA QUE OS DADOS NAO SERAO ARMAZENADOS NO SOFTWARE
tk.Label(frame, text="").grid(row=7, column=0)
tk.Label(frame, text="").grid(row=8, column=0)
tk.Label(frame, text="O usuário e senha não serão armazenados em nenhum local durante a execução do script. Ambos serão utilizados apenas para o acesso ao site.", font=('Arial',15)).grid(row=9, columnspan=2)

label = tk.Label(root, text="Script made by Theo Ravi.", font=('Arial', 10))
label.place(relx=1.0, rely=1.0, anchor='se')

root.mainloop()
#ACESSA O SEI
time.sleep(1)
navegador.find_element(By.XPATH,'//*[@id="sbmAcessar"]').click()    
# ENTRAR NO PROCESSO
navegador.find_element(By.XPATH,'//*[@id="divFiltro"]/div[2]/a').click()


In [ ]:
tbody = navegador.find_element(By.XPATH,'//*[@id="tblProcessosRecebidos"]/tbody')
trs = tbody.find_elements(By.TAG_NAME, 'tr')
lista_processos = []
lista_procConformes = []
for tr in trs:
    tds = tr.find_elements(By.TAG_NAME, 'td')
    for td in tds:
        texto = td.text
        if '.estagio' not in texto:
            texto = texto.replace('Recebidos', '')
            texto = texto.replace(' ', '')
            anotacoes = tr.find_elements(By.XPATH, './/img[@src="svg/anotacao1.svg?11"]')
            if not anotacoes:  # se a lista de anotações estiver vazia, o elemento não existe
                lista_processos.append(texto)
            else:
                lista_procConformes.append(texto)

lista_processos = [processo for processo in lista_processos if processo]
lista_processos.reverse()
lista_procConformes = [processo for processo in lista_procConformes if processo]
lista_procConformes.reverse()

print(lista_procConformes, lista_processos)
print("Quantidade de processos para analisar",len(lista_processos))
print("Quantidade de processos para concluir", len(lista_procConformes))

In [ ]:
lista_procConformes = ["53500.074124/2024-32", "53500.073842/2024-91"]

In [ ]:
#FUNCAO PARA ANALISAR PROCESSO
#VARIAVEL UTILIZADA PARA O SELENIUM RETORNAR PARA A JANELA PRINCIPAL
janela_principal = navegador.current_window_handle
try:
    #FAZ UM LOG DO PROCESSO
    escrever_informacoes(processo, nomeEstag)
    #INICIA A VARIAVEL ZERADA
    impProp=''
    situacao=''
    #PESQUISA NUMERO DO PROCESSO NA CAIXA DE PESQUISA DO SEI PARA ACESSAR O PROCESSO
    navegador.switch_to.default_content()
    navegador.find_element(By.ID,'txtPesquisaRapida').send_keys(processo) 
    elementos = navegador.find_element(By.ID,'txtPesquisaRapida')
    elementos.send_keys(Keys.ENTER)    
    #TEMPO DEFINIDO PARA QUE A PAGINA CARREGUE
    time.sleep(1)
    #ENTRA NO FRAME QUE CONTÉM OS DOCUMENTOS DOS PROCESSOS
    #A PAGINA DOS PROCESSOS SAO DIVIDIDOS EM FRAMES
    navegador.switch_to.frame('ifrArvore')
    #VERIFICA SE O PROCESSO JA FOI DESPACHADO, CASO TENHA SIDO ELE PULA O PROCESSO
    if check_element_exists(By.PARTIAL_LINK_TEXT, 'Despacho Decisório', navegador):
        print(f"O processo {processo} já foi despachado!")
    else:
        #VERIFICA SE HÁ PASTAS NO PROCESSO, SE HOUVER ELE ABRE A PRIMEIRA PARA ENCONTRAR A DECLARACAO DE CONFORMIDADE
        if check_element_exists(By.XPATH, '//*[@id="spanPASTA1"]', navegador):
            navegador.find_element(By.XPATH, '//*[@id="spanPASTA1"]').click()
            time.sleep(1)
        #ENCONTRAR DECLARAÇÃO DE CONFORMIDADE NO PROCESSO DRONE
        if check_element_exists(By.PARTIAL_LINK_TEXT, "Declaração de Conformidade - Drone", navegador):
            #ACESSA O RECIBO PARA PEGAR NOME DO SOLICITANTE
            navegador.find_element(By.PARTIAL_LINK_TEXT, 'Recibo Eletrônico').click()
            navegador.switch_to.default_content()
            #ENTRA NOS FRAMES QUE POSSUE O NOME DO SOLICITANTE
            navegador.switch_to.frame('ifrVisualizacao')
            navegador.switch_to.frame('ifrArvoreHtml')
            #ARMAZENA NOME DO SOLICITANTE NA VARIAVEL
            nomeSol = navegador.find_element(By.XPATH, '//*[@id="conteudo"]/table/tbody/tr[1]/td[2]').text
            #RETORNA AO FRAME DEFAULT
            navegador.switch_to.default_content()
            #COLETAR DADOS DA DECLARAÇÃO DE CONFORMIDADE
            #RETORNA AO FRAME QUE CONTÉM OS DOCUMENTOS
            navegador.switch_to.frame('ifrArvore')
            #CLICA NA DECLARACAO DE CONFORMIDADE
            navegador.find_element(By.PARTIAL_LINK_TEXT, "Declaração de Conformidade - Drone").click()
            time.sleep(1)
            #RETORNA AO FRAME DEFAULT E ENTRA NOS FRAMES QUE CONTÉM A VISUALIZACAO DA DECLARACAO
            navegador.switch_to.default_content()
            navegador.switch_to.frame('ifrVisualizacao')
            navegador.switch_to.frame('ifrArvoreHtml')
            #EXIBE INFORMACOES DO PROCESSO
            print("--------------------------------------------------------------------------")
            print('--------------------------------------------------------------------------')
            print(f"Processo: {processo}")

            #COLETAR TEXTO NOME DO INTERESSADO
            nome_interessado = navegador.find_element(By.XPATH, '/html/body/table[1]/tbody/tr/td').text
            #VERIFICA SE O CAMPO ESTÁ VAZIO, CASO ESTEJA DEFINIRA CM NAO INFORMADO
            if not nome_interessado.strip():
                nome_interessado = 'Não informado'
                print("NOME DO INTERESSADO NÃO INFORMADO")
            #EXIBE NOMES DO INTERESSADO E SOLICITANTE
            print("Interessado: ",nome_interessado)
            print("Solicitante: ",nomeSol)
            print('\n')

            #CHECKBOX DE QUEM ESTÁ FAZENDO O PETICIONAMENTO
            checkbox=navegador.find_element(By.XPATH,'/html/body/table[2]/tbody/tr[1]/td[1]').text
            checkbox2=navegador.find_element(By.XPATH, '/html/body/table[2]/tbody/tr[2]/td[1]').text
            #VERIFICA CAMPO DA CHECKBOX
            #SE OS DOIS CAMPOS ESTIVEREM VAZIOS AVISA QUE NAO FOI INFORMADO QUEM ESTÁ FAZENDO O PETICIONAMENTO
            if not checkbox.strip() and not checkbox2.strip():
                print("O usuário não informou quem está fazendo o peticionamento, confira os nomes do documento.")
            elif not checkbox.strip():
                procuracao_eletronica = navegador.find_element(By.XPATH, '/html/body/table[2]/tbody/tr[2]/td[3]').text
                print("Peticionamento representando terceiro, Pessoa Jurídica ou Pessoa Física.", f"\nProcuração eletrônica: {procuracao_eletronica}.")
            else:
                print("Peticionamento em interesse próprio como Pessoa Física.")
            print('\n')

            #CONTA QUANTIDADE DE LINHAS DA TABELA SOBRE O PRODUTO
            quantidade_linhas = len(navegador.find_elements(By.XPATH, '/html/body/table[3]/tbody/tr'))

            #VERIFICA CAMPOS QUE CONTÉM AS INFORMAÇÕES SOBRE O PRODUTO
            #ARMAZENA OS MODELOS INFORMADOS PARA VERIFICAR SE ESTÃO NA PLANILHA DE DRONES CONFORMES
            modelos = []
            for i in range(1, quantidade_linhas, 1):
                linha = navegador.find_element('xpath', f'/html/body/table[3]/tbody/tr[{i}]')
                celulas = linha.find_elements(By.TAG_NAME,'td')
                modelos_produto = [celula.text for celula in celulas]
                modelos.append(modelos_produto)

            #CRIA DATAFRAME PARA PRINTAR AS INFORMAÇÕES DO PRODUTO JÁ TRATADAS
            df = pd.DataFrame(modelos)
            df.columns = ['Modelo', 'Nome Comercial', 'Número de Série (incluindo rádio controle e óculos)']
            df = df[1:]
            df = df.replace(' ', np.nan)
            df = df.dropna(how='all')
            data = df.values.tolist()
            headers = df.columns.tolist()
            print(tabulate(data, headers=headers, tablefmt='pretty'))
            print('\n')

            #VERIFICA SE O MODELO DO DRONE E RADIO CONTROLE ESTA NA PLANILHA DE DRONES CONFORMES
            modelos = df['Modelo']
            modelos = modelos.reset_index(drop=True)
            checkexcel = verifica_conformidade(modelos, tabela_modelos)
            
            for i in range(len(checkexcel)):
                if checkexcel[i] == True:
                    print(f"O modelo {modelos[i]} está na lista de drones conformes.")
                else:
                    print(f"O modelo {modelos[i]} não se encontra na lista de drones conformes")
            print('\n')
            #EXIBE CÓDIGO DE RAST"REIO
            #LINHA ESPECIFICA ESCREVE NA PLANILHA SE O PRODUTO ESTA RETIDO E, CASO ESTEJA, ESCREVE O CODIGO DE RASTREIO
            n_serie = navegador.find_element(By.XPATH, '/html/body/table[3]/tbody/tr[2]/td[3]').text
            codigo_rastreio = navegador.find_element(By.XPATH,'/html/body/table[4]/tbody/tr/td[2]').text
            #VERIFICA SE HÁ ALGUM TEXTO NA TABELA COM O CÓDIGO DE RASTREIO
            #SE HOUVER ALGUM TEXTO ELE DA COMO RETIDO, CASO NAO TENHA TEXTO ELE DA COMO NAO RETIDO
            if not codigo_rastreio.strip():
                print("Produto não retido ou código de rastreio não informado.")
                retido='Não'
            else:
                if not n_serie.strip():
                    print(f'O código de rastreio é: {codigo_rastreio}.')
                    retido='Sim'
                elif n_serie.strip():
                    print('Confira se o produto está retido.')
                    verifica_retido = int(input('O produto está retido? Se sim digite [1], se não digite [2]: '))
                    if verifica_retido == 1:
                        retido='Sim'
                    elif verifica_retido == 2:
                        retido='Não'
            print('\n')
            print('Confira o relatório fotográfico e veja se os documentos estão conformes!\n')
            print('--------------------------------------------------------------------------')
            print('--------------------------------------------------------------------------')
            
        #ENCONTRAR SE HÁ ALGUMA PASTA DE DOCUMENTOS OU DECLARAÇÃO DE CONFORMIDADE NO PROCESSO
        elif check_element_exists(By.PARTIAL_LINK_TEXT, "Declaração de Conformidade - Importado Uso Próprio", navegador) or check_element_exists(By.XPATH, '//*[@id="spanPASTA1"]', navegador):
            #PEGAR NOME DO SOLICITANTE NO RECIBO ELETRONICO
            navegador.find_element(By.PARTIAL_LINK_TEXT, 'Recibo Eletrônico').click()
            navegador.switch_to.default_content()
            time.sleep(0.5)
            navegador.switch_to.frame('ifrVisualizacao')
            navegador.switch_to.frame('ifrArvoreHtml')
            nomeSol = navegador.find_element(By.XPATH, '//*[@id="conteudo"]/table/tbody/tr[1]/td[2]').text
            navegador.switch_to.default_content()
            #CLICA NA DECLARACAO DE CONFORMIDADE
            #COLETAR DADOS DA DECLARAÇÃO DE CONFORMIDADE
            navegador.switch_to.frame('ifrArvore')
            navegador.find_element(By.PARTIAL_LINK_TEXT, "Declaração de Conformidade - Importado Uso Próprio").click()
            time.sleep(0.5)
            navegador.switch_to.default_content()
            navegador.switch_to.frame('ifrVisualizacao')
            navegador.switch_to.frame('ifrArvoreHtml')

            #EXIBE INFORMACOES DO PROCESSO
            print("--------------------------------------------------------------------------")
            print('--------------------------------------------------------------------------')
            print(f"Processo: {processo}")

            #COLETAR TEXTO NOME DO INTERESSADO
            nome_interessado=navegador.find_element(By.XPATH, '/html/body/table[1]/tbody/tr/td/p').text
            if not nome_interessado.strip():
                nome_interessado = 'Não informado'
                print("NOME DO INTERESSADO NÃO INFORMADO")
            print(nome_interessado)
            print(nomeSol)
            print('\n')
            #CHECKBOX DE QUEM ESTÁ FAZENDO O PETICIONAMENTO
            checkbox=navegador.find_element('xpath','/html/body/table[2]/tbody/tr[1]/td[1]/p').text
            checkbox2=navegador.find_element(By.XPATH, '/html/body/table[2]/tbody/tr[2]/td[1]').text

            #CHECA CAMPO DA CHECKBOX
            if not checkbox.strip() and not checkbox2.strip():
                print("O usuário não informou quem está fazendo o peticionamento, confira os nomes do documento.")
            elif not checkbox.strip():
                procuracao_eletronica = navegador.find_element(By.XPATH, '/html/body/table[2]/tbody/tr[2]/td[3]').text
                print("Peticionamento representando terceiro, Pessoa Jurídica ou Pessoa Física.", f"\nProcuração eletrônica: {procuracao_eletronica}.")
            else:
                print("Peticionamento em interesse próprio como Pessoa Física.")
            print('\n')

            #INFORMA O TIPO DE PRODUTO QUE ESTÁ SENDO TRATADO
            print("\nTipo de produto:")
            quantidade_linhasPro = len(navegador.find_elements(By.XPATH, '/html/body/table[3]/tbody/tr'))
            tipoPro = []
            for i in range(1, quantidade_linhasPro, 1):
                linhas = navegador.find_element(By.XPATH, f'/html/body/table[3]/tbody/tr[{i}]')
                celulas2 = linhas.find_elements(By.TAG_NAME,'td')
                produtos = [celula2.text for celula2 in celulas2]
                tipoPro.append(produtos)
            dfPro = pd.DataFrame(tipoPro)
            dfPro.columns = ['Tipo de Equipamento', 'Assinale com um X', 'Observação:']
            dfPro = dfPro.drop('Observação:', axis=1)
            dfPro = dfPro[1:]

            #EXIBE A TABELA DO TIPO DE PRODUTO
            data = dfPro.values.tolist()
            headers = dfPro.columns.tolist()
            print(tabulate(data, headers=headers, tablefmt='pretty'))
            print('\n')

            #CONTA QUANTIDADE DE LINHAS DA TABELA SOBRE O PRODUTO
            quantidade_linhas = len(navegador.find_elements(By.XPATH, '/html/body/table[4]/tbody/tr'))

            #VERIFICA CAMPOS QUE CONTÉM AS INFORMAÇÕES SOBRE O PRODUTO
            modelos = []
            for i in range(1, quantidade_linhas, 1):
                linha = navegador.find_element('xpath', f'/html/body/table[4]/tbody/tr[{i}]')
                celulas = linha.find_elements(By.TAG_NAME,'td')
                modelos_produto = [celula.text for celula in celulas]
                modelos.append(modelos_produto)
            
            #CRIA DATAFRAME E VERIFICA SE OS MODELOS FORNECIDOS ESTÃO NA PLANILHA DE DRONES CONFORMES
            df = pd.DataFrame(modelos)
            df.columns = ['Modelo', 'Nome Comercial', 'Número de Série']
            df = df[1:]
            df = df.replace(' ', np.nan)
            df = df.dropna(how='all')
            modelos = df['Modelo']
            modelos = modelos.reset_index(drop=True)
            checkexcel = verifica_conformidade(modelos, tabela_modelos)                        
            print("Modelos:")
            #CRIA DATAFRAME PARA PRINTAR AS INFORMAÇÕES DO PRODUTO
            data2 = df.values.tolist()
            headers2 = df.columns.tolist()
            print(tabulate(data2, headers=headers2, tablefmt='pretty'))
            print('\n')

            for i in range(len(checkexcel)):
                if checkexcel[i] == True:
                    print(f"O modelo {modelos[i]} está na lista de drones conformes.")
                else:
                    print(f"O modelo {modelos[i]} não se encontra na lista de drones conformes")
            print('\n')
            #EXIBE CÓDIGO DE RASTREIO
            #ESCREVE NA TABELA EXCEL SE ESTA RETIDO, CASO ESTEJA INSERE CODIGO DE RASTREIO
            n_serie = navegador.find_element(By.XPATH, '/html/body/table[4]/tbody/tr[2]/td[3]').text
            codigo_rastreio = navegador.find_element(By.XPATH,'/html/body/table[5]/tbody/tr/td[2]').text
            if not codigo_rastreio.strip():
                print("Produto não retido ou código de rastreio não informado.")
                retido='Não'
            else:
                if not n_serie.strip():
                    print(f'O código de rastreio é: {codigo_rastreio}.')
                    retido='Sim'
                elif n_serie.strip():
                    print('Confira se o produto está retido.')
                    verifica_retido = int(input('O produto está retido? Se sim digite [1], se não digite [2]: '))
                    if verifica_retido == 1:
                        retido='Sim'
                    elif verifica_retido == 2:
                        retido='Não'
            print('\n')
            print('Confira o relatório fotográfico e veja se os documentos estão conformes!\n')
            print('--------------------------------------------------------------------------')
            print('--------------------------------------------------------------------------')
            
        time.sleep(1)
        navegador.switch_to.default_content()

        #CRIA DESPACHO DECISORIO
        #PEDE PRA USUARIO VERIFICAR SE ESTÁ TUDO CERTO
        #VOLTA PARA A PAGINA DE INICIO DO PROCESSO
        while True:
            try:
                confirmacao = int(input('Caso esteja tudo certo e NÃO HAJA UM DESPACHO JÁ CRIADO digite [1] para continuar, senão, digite [2]: '))
                if (nome_interessado == 'Não informado' or '') and confirmacao == 1:
                    print("Nome do interessado não informado, não é possível criar despacho, selecione outra opção!")
                elif confirmacao == 1:
                    break
                elif confirmacao == 2:
                    break
                else:
                    print("Opção inválida, tente novamente!")
            except ValueError:
                print("Opção inválida, tente novamente!")
        if confirmacao == 1:
            navegador.switch_to.frame('ifrArvore')
            arvoredocs = navegador.find_elements(By.CLASS_NAME, 'infraArvoreNo')

            #VERIFICA SE O DOCUMENTO ESTÁ RESTRITO, SE ESTIVER ELE IRÁ DEIXAR COMO PUBLICO UTILIZANDO O SIMBOLO DE RESTRITO COMO REFERENCIA
            elementos_com_src = navegador.find_elements(By.CSS_SELECTOR, "[src='svg/processo_restrito.svg?11']")
            #CRIA UMA LISTA PARA OS DOCUMENTOS QUE DEVERÃO SER DEIXADOS COMO PUBLICO
            elementos_para_clicar = []
            #ITERA SOBRE OS DOCUMENTOS 
            for doc in arvoredocs:
                #ITERA SOBRE OS ELEMENTOS COM O SIMBOLO DE RESTRITO
                for elemento in elementos_com_src:
                    #VERIFICA SE O DOCUMENTO ESTÁ AO LADO DO SIMBOLO DE RESTRITO
                    if elemento in doc.find_elements(By.XPATH, 'following-sibling::*[2]/*'):
                        #SE O SIMBOLO DE RESTRITO ESTIVER NO DOCUMENTO ELE INCLUIRÁ NA LISTA DE DOCUMENTOS A SEREM ACESSADOS
                        elementos_para_clicar.append(doc.get_attribute("id"))
                        break

            #AGORA COM O ID DO DOCUMENTO ELE CLICARÁ EM CADA DOCUMENTO RESTRITO PARA DEIXAR PUBLICO
            for id in elementos_para_clicar:
                #ENCONTRA O ELEMENTO PARA CLICAR
                doc = navegador.find_element(By.ID, id)
                doc.click()
                time.sleep(0.7)
                navegador.switch_to.default_content()
                navegador.switch_to.frame('ifrVisualizacao')
                #CLICA NO SIMBOLO DE ALTERAR DOCUMENTO
                navegador.find_element(By.XPATH,'//*[@id="divArvoreAcoes"]/a[2]').click()
                #SELECIONA A OPCAO DE PUBLICO NO DOCUMENTO
                navegador.find_element(By.XPATH,'//*[@id="divOptPublico"]/div/label').click()
                #SALVA AS MUDANCAS
                navegador.find_element(By.ID,'btnSalvar').click()
                navegador.switch_to.default_content()
                navegador.switch_to.frame('ifrArvore')
            navegador.switch_to.default_content()
            navegador.switch_to.frame('ifrArvore')
            #ENTRA NA PAGINA INICIAL DO PROCESSO
            #VERIFICA SE E UM PROCESSO DE DRONE OU IMPORTADO PARA USO PROPRIO
            if check_element_exists(By.PARTIAL_LINK_TEXT, "Declaração de Conformidade - Drone", navegador):
                navegador.switch_to.default_content()
                navegador.find_element(By.ID,'txtPesquisaRapida').send_keys(processo)
                elementos = navegador.find_element(By.ID,'txtPesquisaRapida')
                elementos.send_keys(Keys.ENTER)
                #CRIA DESPACHO
                navegador.switch_to.frame('ifrVisualizacao')
                time.sleep(1)
                #CLICA NO INCONE DE INCLUIR DOCUMENTO
                navegador.find_element(By.XPATH,'//*[@id="divArvoreAcoes"]/a[1]').click()
                time.sleep(1)
                #CLICA NA OPCAO DE DESPACHO DECISORIO
                navegador.find_element(By.XPATH,'//*[@id="tblSeries"]/tbody/tr[16]/td/a[2]').click()
                #SELECIONA TEXTO PADRAO
                navegador.find_element(By.XPATH,'//*[@id="divOptTextoPadrao"]/div').click()
                #ENVIA QUAL DESPACHO DECISORIO DEVE SER CRIADO
                navegador.find_element(By.XPATH,'//*[@id="txtTextoPadrao"]').send_keys('Despacho Decisório de Homologação Drones')
            #VERIFICA SE E UM PROCESSO DE DRONE OU IMPORTADO PARA USO PROPRIO
            elif check_element_exists(By.PARTIAL_LINK_TEXT, "Declaração de Conformidade - Importado Uso Próprio", navegador):
                impProp='Sim'
                navegador.switch_to.default_content()
                navegador.find_element(By.ID,'txtPesquisaRapida').send_keys(processo)
                elementos = navegador.find_element(By.ID,'txtPesquisaRapida')
                elementos.send_keys(Keys.ENTER)
                #CRIA DESPACHO
                navegador.switch_to.frame('ifrVisualizacao')
                time.sleep(1)
                #CLICA NO INCONE DE INCLUIR DOCUMENTO
                navegador.find_element(By.XPATH,'//*[@id="divArvoreAcoes"]/a[1]').click()
                time.sleep(1)
                #CLICA NA OPCAO DE DESPACHO DECISORIO
                navegador.find_element(By.XPATH,'//*[@id="tblSeries"]/tbody/tr[16]/td/a[2]').click()
                #SELECIONA TEXTO PADRAO
                navegador.find_element(By.XPATH,'//*[@id="divOptTextoPadrao"]/div').click()
                #ENVIA QUAL DESPACHO DECISORIO DEVE SER CRIADO
                navegador.find_element(By.XPATH,'//*[@id="txtTextoPadrao"]').send_keys('Despacho Decisório de Homologação não licenciados')
            time.sleep(2)
            #CLICA NA PRIMEIRA OPCAO
            navegador.find_element(By.XPATH,'//*[@id="divInfraAjaxtxtTextoPadrao"]/ul/li/a').click()
            #COLOCA O DESPACHO COMO PUBLICO
            navegador.find_element(By.XPATH,'//*[@id="divOptPublico"]/div').click()
            #SALVA DESPACHO
            navegador.find_element(By.ID,'btnSalvar').click()
            time.sleep(0.7)
            try:
                navegador.switch_to.window(navegador.window_handles[-1])
                navegador.close()
                time.sleep(0.7)
                #MUDA PARA JANELA PRINCIAPL DO PROGRAMA
                navegador.switch_to.window(janela_principal)
            except:
                #MUDA PARA JANELA PRINCIAPL DO PROGRAMA
                navegador.switch_to.window(janela_principal)
            #MUDA PARA JANELA PRINCIAPL DO PROGRAMA
            navegador.switch_to.window(janela_principal)
            #INCLUI DESPACHO NO BLOCO
            navegador.switch_to.default_content()
            navegador.switch_to.frame('ifrArvore')
            #CLICA NO DESPACHO DECISORIO
            navegador.find_element(By.PARTIAL_LINK_TEXT, "Despacho Decisório").click()
            navegador.switch_to.default_content()
            navegador.switch_to.frame('ifrVisualizacao')
            time.sleep(1)
            #CLICA NO ICONE DE LEGO
            navegador.find_element(By.XPATH, '//*[@id="divArvoreAcoes"]/a[8]').click()
            #SELECIONA BLOCO (SELECIONA O PRIMEIRO DESPACHO PARA DRONES APROVADOS QUE LER)
            select_element = navegador.find_element(By.ID, 'selBloco')
            select = Select(select_element)
            #PROCURA O PRIMEIRO BLOCO QUE TENHA O TEXTO "Despachos para Drones aprovados"
            for opcao in select.options:
                if "Despachos para Drones aprovados" in opcao.text:
                    select.select_by_visible_text(opcao.text)
                    break
            #CLICA NO BOTAO DE INCLUIR NO BLOCO
            navegador.find_element(By.XPATH, '//*[@id="sbmIncluir"]').click()
            #VOLTA PARA PAGINA INICIAL DO PROCESSO
            navegador.switch_to.default_content()
            navegador.find_element(By.ID,'txtPesquisaRapida').send_keys(processo)
            elementos = navegador.find_element(By.ID,'txtPesquisaRapida')
            elementos.send_keys(Keys.ENTER)
            navegador.switch_to.frame('ifrVisualizacao')
            time.sleep(1)
            #ADICIONA NOTA PARA AGUARDAR ASSINATURA
            #CLICA NO ICONE DE ANOTACAO
            navegador.find_element(By.XPATH, '//*[@id="divArvoreAcoes"]/a[17]').click()
            #INSERE O TEXTO DA ANOTACAO
            navegador.find_element(By.ID, 'txaDescricao').send_keys('Aguardando assinatura.')
            #SALVA O TEXTO
            navegador.find_element(By.NAME, 'sbmRegistrarAnotacao').click()
            #DA COMO APROVADO E ANOTA NO TXT DE PROCESSOS CONFORMES
            situacao = 'Aprovado'
            print("Próximo processo...")

        #CONDICAO DE PROCESSO NAO CONFORME
        else:
            print(f'Tome as medidas necessárias para o processo {processo}.')
            #MOSTRA OPCOES DE EXIGENCIA
            while True:
                try:
                    exigencia = int(input('Se houver alguma exigência digite [1], senão, digite [2]: '))
                    if exigencia == 1:
                        break
                    elif exigencia == 2:
                        break
                    else:
                        print("Opção inválida, tente novamente!")
                except ValueError:
                    print("Opção inválida, tente novamente!")
            #CONDICAO DE EXIGENCIA
            if exigencia == 1:
                navegador.switch_to.default_content()
                navegador.switch_to.frame('ifrArvore')
                #CLICA NA DECLARACAO DE CONFORMIDADE
                navegador.find_element(By.PARTIAL_LINK_TEXT, "Declaração de Conformidade").click()
                time.sleep(1)
                navegador.switch_to.default_content()
                navegador.switch_to.frame('ifrVisualizacao')
                navegador.switch_to.frame('ifrArvoreHtml')
                #ARMAZENA O EMAIL DO SOLICITANTE
                emailSol = navegador.find_element(By.XPATH, '/html/body/div[3]/a[1]').text
                navegador.switch_to.default_content()
                #RETORNA PARA A PAGINA INICIAL DO PROCESSO
                navegador.find_element(By.ID,'txtPesquisaRapida').send_keys(processo)
                elementos = navegador.find_element(By.ID,'txtPesquisaRapida')
                elementos.send_keys(Keys.ENTER)
                time.sleep(1)
                navegador.switch_to.frame('ifrVisualizacao')
                #CLICA NO ICONE DE ENVIO DE EMAIL
                navegador.find_element(By.XPATH, '//*[@id="divArvoreAcoes"]/a[11]').click()
                time.sleep(1)
                #VAI PARA A JANELA MAIS RECENTE ABERTA
                navegador.switch_to.window(navegador.window_handles[-1])
                time.sleep(1)
                #ABRE O DROPDOWN COM AS OPCOES DE EMAIL DA ANATEL
                select_element = navegador.find_element(By.XPATH, '//*[@id="selDe"]')
                select = Select(select_element)
                #SELECIONA O EMAIL DA ANATEL
                select.select_by_visible_text('ANATEL/E-mail de replicação <nao-responda@anatel.gov.br>')
                #ESCREVE O EMAIL DO SOLICITANTE
                navegador.find_element(By.XPATH, '//*[@id="s2id_autogen1"]').send_keys(emailSol)
                time.sleep(1.2)
                #CLICA NO EMAIL DO SOLICITANTE
                navegador.find_element(By.CLASS_NAME, 'select2-result-label').click()
                #MOSTRA AS OPCOES DE EXIGENCIA
                #EXECUTA A CONDICAO PARA CADA EXIGENCIA
                while True:
                    try:
                        print("Selecione o tipo de exigência abaixo:\n",
                            "[1] Falta algum anexo no processo.\n",
                            "[2] Erro na declaração de conformidade ou alguma exigência que é necessário abrir novo processo.\n",
                            "[3] Outros.")
                        time.sleep(0.5)
                        exig = int(input("Opção: "))
                    except ValueError:
                        print("Opção inválida. Digite um número inteiro entre 1 e 3.")
                        continue
                    if exig in range(1, 4):
                        if exig == 1:
                            #PUXA FUNCAO INSIRA_ANEXO CONTENDO O CORPO DE EMAIL PEDINDO ANEXOS
                            insira_anexo(processo, navegador)
                            #PEDE PARA USUARIO CONFERIR SE ESTÁ TUDO CERTO COM O EMAIL
                            while True:
                                try:
                                    confere = int(input('Caso esteja tudo certo digite [1], caso tenha algum erro digite [2]: '))
                                    if confere == 1:
                                        break
                                    elif confere == 2:
                                        break
                                    else:
                                        print("Opção inválida, tente novamente!")
                                except ValueError:
                                    print("Opção inválida, tente novamente!")
                            if confere == 1:
                                #ENVIA EMAIL COM A EXIGENCIA
                                navegador.find_element(By.XPATH, '//*[@id="divInfraBarraComandosInferior"]/button[1]').click()
                                time.sleep(0.5)
                                #FECHA ALERTA DO NAVEGADOR
                                alert = Alert(navegador)
                                alert.accept()
                                time.sleep(0.5)
                                #VOLTA O FOCO DO SELENIUM PARA A PAGINA PRINCIPAL NOVAMENTE
                                navegador.switch_to.window(janela_principal)
                                navegador.switch_to.default_content()
                                #VOLTA PARA A PAGINA INICIAL DO PROCESSO
                                navegador.find_element(By.ID,'txtPesquisaRapida').send_keys(processo) 
                                elementos = navegador.find_element(By.ID,'txtPesquisaRapida')
                                elementos.send_keys(Keys.ENTER)    
                                time.sleep(1)
                                #INSERE TAG REFERENTE A PROCESSOS INTERCORRENTES
                                navegador.switch_to.frame('ifrVisualizacao')
                                #CLICA NO ICONE DE TAG
                                time.sleep(0.5)
                                navegador.find_element(By.XPATH, '//*[@id="divArvoreAcoes"]/a[25]').click()
                                try:    
                                    navegador.find_element(By.XPATH, '//*[@id="selMarcador"]/div/a').click()
                                except:
                                    navegador.find_element(By.XPATH, '//*[@id="tblMarcadores"]/tbody/tr[2]/td[6]/a[2]/img').click()
                                    alert.accept()
                                    navegador.find_element(By.XPATH, '//*[@id="btnAdicionar"]').click()
                                    time.sleep(0.5)
                                    navegador.find_element(By.XPATH, '//*[@id="selMarcador"]/div/a').click()
                                
                                time.sleep(0.5)
                                #CLICA NA TAG DE INTERCORRENTE
                                navegador.find_element(By.XPATH, '//*[@id="selMarcador"]/ul/li[18]/a').click()
                                #PEDE O TEXTO DA TAG
                                textoTag = input("Insira o texto da tag: ")
                                textoFinaltag = nomeEstag+"\n"+textoTag
                                #COLOCA O TEXTO NA TAG
                                navegador.find_element(By.XPATH, '//*[@id="txaTexto"]').send_keys(textoFinaltag)
                                #SALVA TAG
                                navegador.find_element(By.XPATH, '//*[@id="sbmSalvar"]').click()
                                #DEFINE SITUACAO DO PROCESSO
                                situacao = 'Intercorrente'
                            else:
                                #CANCELA ENVIO DO EMAIL SE HOUVER ALGUM ERRO
                                print("Faça as alterações necessárias mais tarde.")
                                navegador.find_element(By.XPATH, '//*[@id="btnCancelar"]').click()
                                navegador.switch_to.window(janela_principal)
                                return
                        elif exig == 2:
                            #PUXA FUNCAO ERRO_DECLARACAO CONTENDO O CORPO DE EMAIL PEDINDO PARA ABRIR NOVO PROCESSO SEI
                            erro_declaracao(processo, impProp, navegador)
                            #PEDE PARA USUARIO CONFERIR O EMAIL
                            while True:
                                try:
                                    confere = int(input('Caso esteja tudo certo digite [1], caso tenha algum erro digite [2]: '))
                                    if confere == 1:
                                        break
                                    elif confere == 2:
                                        break
                                    else:
                                        print("Opção inválida, tente novamente!")
                                except ValueError:
                                    print("Opção inválida, tente novamente!")
                            if confere == 1:
                                #ENVIA O EMAIL COM EXIGENCIA
                                navegador.find_element(By.XPATH, '//*[@id="divInfraBarraComandosInferior"]/button[1]').click()
                                time.sleep(0.5)
                                #FECHA ALERTA DO NAVEGADOR
                                alert = Alert(navegador)
                                alert.accept()
                                time.sleep(0.5)
                                #RETORNA O FOCO PARA A JANELA PRINCIPAL
                                navegador.switch_to.window(janela_principal)
                                navegador.switch_to.default_content()
                                #VOLTA PARA PAGINA INICIAL DO PROCESSO
                                navegador.find_element(By.ID,'txtPesquisaRapida').send_keys(processo) 
                                elementos = navegador.find_element(By.ID,'txtPesquisaRapida')
                                elementos.send_keys(Keys.ENTER)    
                                time.sleep(1)
                                #INSERE TAG REFERENTE A PROCESSOS INTERCORRENTES 
                                navegador.switch_to.frame('ifrVisualizacao')
                                #CLICA NO ICONE DE TAG
                                time.sleep(0.5)
                                navegador.find_element(By.XPATH, '//*[@id="divArvoreAcoes"]/a[25]').click()
                                try:    
                                    navegador.find_element(By.XPATH, '//*[@id="selMarcador"]/div/a').click()
                                except:
                                    navegador.find_element(By.XPATH, '//*[@id="tblMarcadores"]/tbody/tr[2]/td[6]/a[2]/img').click()
                                    alert.accept()
                                    navegador.find_element(By.XPATH, '//*[@id="btnAdicionar"]').click()
                                    time.sleep(0.5)
                                    navegador.find_element(By.XPATH, '//*[@id="selMarcador"]/div/a').click()
                                
                                #CLICA NO DROPDOWN COM OS TIPOS DE TAG
                                time.sleep(0.5)
                                #CLICA NA TAG DE PENDENCIA
                                navegador.find_element(By.XPATH, '//*[@id="selMarcador"]/ul/li[19]/a').click()
                                #PEDE O TEXTO DA TAG
                                textoTag = input("Insira o texto da tag: ")
                                textoFinaltag = nomeEstag+"\n"+textoTag
                                navegador.find_element(By.XPATH, '//*[@id="txaTexto"]').send_keys(textoFinaltag)
                                #SALVA A TAG
                                navegador.find_element(By.XPATH, '//*[@id="sbmSalvar"]').click()
                                #DEFINE SITUACAO DO PROCESSO
                                situacao = 'Exigência'
                            else:
                                #CANCELA O ENVIO CASO TENHA ALGUM ERRO
                                print("Faça as alterações necessárias mais tarde.")
                                navegador.find_element(By.XPATH, '//*[@id="btnCancelar"]').click()
                                navegador.switch_to.window(janela_principal)
                                return
                        elif exig == 3:
                            #PUXA FUNCAO OUTRO_ERRO EM QUE USUARIO INSERE O EMAIL COMO QUISER
                            outro_erro(navegador)
                            #PEDE PARA USUARIO ESCREVER EMAIL E DIGITAR 1 PARA CONTINUAR
                            while True:
                                try:
                                    confere = int(input('Escreva o email da forma que deseja e logo em seguida digite [1], caso queira cancelar digite [2]: '))
                                    if confere == 1:
                                        break
                                    elif confere == 2:
                                        break
                                    else:
                                        print("Opção inválida, tente novamente!")
                                except ValueError:
                                    print("Opção inválida, tente novamente!")
                            if confere == 1:
                                #ENVIA EMAIL
                                navegador.find_element(By.XPATH, '//*[@id="divInfraBarraComandosInferior"]/button[1]').click()
                                time.sleep(0.5)
                                #FECHA ALERTA DO NAVEGADOR
                                alert = Alert(navegador)
                                alert.accept()
                                time.sleep(0.5)
                                #RETORNA O FOCO PARA A JANELA PRINCIPAL
                                navegador.switch_to.window(janela_principal)
                                navegador.switch_to.default_content()
                                #VAI PARA A JANELA INICIAL DO PROCESSO
                                navegador.find_element(By.ID,'txtPesquisaRapida').send_keys(processo) 
                                elementos = navegador.find_element(By.ID,'txtPesquisaRapida')
                                elementos.send_keys(Keys.ENTER)    
                                time.sleep(1)
                                navegador.switch_to.frame('ifrVisualizacao')
                                #CLICA NO ICONE DE TAG
                                time.sleep(0.5)
                                navegador.find_element(By.XPATH, '//*[@id="divArvoreAcoes"]/a[25]').click()
                                try:    
                                    navegador.find_element(By.XPATH, '//*[@id="selMarcador"]/div/a').click()
                                except:
                                    navegador.find_element(By.XPATH, '//*[@id="tblMarcadores"]/tbody/tr[2]/td[6]/a[2]/img').click()
                                    alert.accept()
                                    navegador.find_element(By.XPATH, '//*[@id="btnAdicionar"]').click()
                                    time.sleep(0.5)
                                    navegador.find_element(By.XPATH, '//*[@id="selMarcador"]/div/a').click()
                                
                                time.sleep(0.5)
                                #INSERE TAG DE 'PENDENCIAS'
                                navegador.find_element(By.XPATH, '//*[@id="selMarcador"]/ul/li[19]/a').click()
                                #PEDE TEXTO DA TAG
                                textoTag = input("Insira o texto da tag: ")
                                #INSERE O TEXTO DA TAG
                                navegador.find_element(By.XPATH, '//*[@id="txaTexto"]').send_keys(textoTag)
                                #SALVA TAG
                                navegador.find_element(By.XPATH, '//*[@id="sbmSalvar"]').click()
                                #DEFINE SITUACAO DO PROCESSO
                                situacao = 'Exigência'
                            else:
                                #CANCELA ENVIO DO PROCESSO
                                print("Faça as alterações necessárias mais tarde.")
                                navegador.find_element(By.XPATH, '//*[@id="btnCancelar"]').click()
                                navegador.switch_to.window(janela_principal)
                                return
                        #VOLTA PARA A PAGINA INICIAL DO PROCESSO
                        navegador.switch_to.default_content()
                        navegador.find_element(By.ID,'txtPesquisaRapida').send_keys(processo) 
                        elementos = navegador.find_element(By.ID,'txtPesquisaRapida')
                        elementos.send_keys(Keys.ENTER)
                        time.sleep(1)
                        navegador.switch_to.frame('ifrVisualizacao')
                        #CLICA NO ICONE DE CONCLUIR PROCESSO
                        navegador.find_element(By.XPATH, '//*[@id="divArvoreAcoes"]/a[20]').click()
                        print("Próximo processo...")
                        break
                    #CONDICAO PARA OPCAO INEXISTENTE
                    else:
                        print("Opção inválida. Tente novamente.")
            #CONDICAO CASO PRECISE PULAR O PROCESSO
            else:
                print("Pulando processo.")
        #VERIFICA SE O PROCESSO CONTEM ALGUMA SITUACAO ANTES DE ESCREVER SOBRE
        #SE HOUVER ALGUM ERRO OU PULAR PROCESSO NAO ESCREVERA NA PLANILHA
        if situacao == 'Aprovado' or situacao == 'Exigência' or situacao == 'Intercorrente':
            preenche_planilhageral(processo, nomeEstag, retido, situacao, codigo_rastreio, nome_interessado)
        else:
            print('--------------------------------------------------------------------------')
            print('--------------------------------------------------------------------------')
except Exception as error:
    print(f"Ocorreu um erro: {error}")
pyautogui.PAUSE = 0.7